In [1]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import matplotlib.pyplot as plt
import time
import math
import pandas as pd
import tqdm

import network

In [41]:

N_USERS = 10
M_POSES = 256  # Number of points to sample joint-space
K_TASK_POINTS = 256   # Number of points to generate in task-space
LATENT_DIM = 32
LEARNING_RATE = 1e-4
EPOCHS = 1000
BATCH_SIZE = 256
BETA_KL = 0.001 # Weight for KL loss

DATASET_SIZE = 2560  # Total number of samples in dataset

# Setup device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

joint_limits = np.loadtxt("joint_limits.csv", delimiter=',', skiprows=1)
joint_limits = torch.tensor(joint_limits, dtype=torch.float32)

joint_configs = np.loadtxt("all_users_fROM_data.csv", delimiter=',', skiprows=1)
joint_configs = torch.tensor(joint_configs, dtype=torch.float32)




Using device: cuda


In [42]:
joint_configs = joint_configs.view(-1, M_POSES, 5)  # [N, M, 4]
joint_configs = joint_configs[:, :, 1:]  # Remove user ID column

joint_limits = joint_limits[:, 1:]  # Remove user ID column

In [43]:
print(joint_limits.shape)
print(joint_limits[:3])
print(joint_configs.shape)
print(joint_configs[:3])


dataset = TensorDataset(joint_configs, joint_limits)

dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

model = network.fROM_VAE_joints(joint_dim=4, latent_dim=LATENT_DIM, global_feat_dim=1024, output_limits_dim=8).to(device)

optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
recon_loss_fn = nn.MSELoss()

torch.Size([10, 8])
tensor([[-180.,   50.,  -90.,  180.,  -90.,   90.,    0.,  150.],
        [-180.,   50.,  -90.,  180.,  -90.,   90.,    0.,   90.],
        [-180.,   50.,  -90.,  180.,  -90.,   90.,    0.,   40.]])
torch.Size([10, 256, 4])
tensor([[[ 8.7266e-01,  1.6639e-01, -2.8156e-05,  2.3034e-01],
         [ 8.7266e-01, -3.2207e-01,  9.3389e-06,  7.9934e-01],
         [ 8.7266e-01, -1.9184e-01, -4.1759e-05,  2.2290e-01],
         ...,
         [-1.8435e-01, -4.6623e-01,  8.4480e-02,  1.6179e+00],
         [ 4.6581e-01, -2.1951e-01,  7.9893e-02,  2.4813e+00],
         [ 8.7266e-01, -2.4470e-01, -3.6765e-05,  6.9548e-01]],

        [[ 4.7058e-01, -7.3137e-01,  4.9652e-01,  1.5708e+00],
         [ 8.7266e-01,  9.2328e-02, -7.7067e-06,  5.4988e-01],
         [-1.5017e-02,  8.7780e-02,  2.1368e-02,  1.0579e+00],
         ...,
         [ 4.0863e-02,  5.5301e-01, -6.5348e-02,  1.0296e+00],
         [ 6.4327e-01,  9.1839e-01,  6.2848e-02,  1.1916e+00],
         [ 5.9011e-01, -7.2375e-0

In [36]:
b = iter(dataloader)
x = next(b)
print(x.shape)
b = iter(dataloader)
x = next(b)

print(x.shape)
len(dataloader)

AttributeError: 'list' object has no attribute 'shape'

In [ ]:
from tkinter import Y


for epoch in range(EPOCHS):
    model.train()
    total_loss_epoch = 0
    total_recon_loss_epoch = 0
    total_kl_loss_epoch = 0
    
    for x, y in dataloader: 
        x = x.to(device)  # [B, M, 4]
        y = y.to(device)  # [B, 8]
        
        # Forward pass
        y_recon, mu, logvar = model(x) # [B, 8], [B, D_z], [B, D_z]
        
        # Loss
        # Reconstruction Loss (MSE on the 8 limit values)
        recon_loss = recon_loss_fn(y_recon, y)
        
        # KL
        kl_loss = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
        
        loss = (recon_loss / BATCH_SIZE) + (BETA_KL * kl_loss / BATCH_SIZE)
        
        if epoch == EPOCHS - 400:
            print(f"{y_recon[:3].detach().cpu().numpy()}")
            print(f"{y[:3].detach().cpu().numpy()}")
            print(f"recon_loss: {recon_loss.item():.4f}, kl_loss: {kl_loss.item():.4f}, total_loss: {loss.item():.4f}")
            
        
        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        total_loss_epoch += loss.item() * BATCH_SIZE
        total_recon_loss_epoch += recon_loss.item()
        total_kl_loss_epoch += kl_loss.item()

    # Print epoch stats
    avg_loss = total_loss_epoch / DATASET_SIZE  # 2560
    avg_recon = total_recon_loss_epoch / DATASET_SIZE
    avg_kl = total_kl_loss_epoch / DATASET_SIZE
    
    # if (epoch + 1) % 20 == 0:
    #     print(f"Epoch [{epoch+1}/{EPOCHS}], Loss: {avg_loss:.4f}, "
    #           f"Recon Loss (MSE): {avg_recon:.4f}, KL Loss: {avg_kl:.4f}")

[[-1.7981667e+02  4.9289745e+01 -8.9408440e+01  1.7771039e+02
  -9.0076645e+01  9.0120239e+01 -4.8815664e-02  8.9359619e+01]
 [-1.7785178e+02  4.9897934e+01 -8.9843361e+01  1.8046782e+02
  -3.0145691e+01  3.0245142e+01 -1.1258405e-02  9.1110390e+01]
 [-1.7502805e+02  5.0140553e+01 -4.9906029e+01  1.0185955e+02
  -9.1283020e+01  9.1505585e+01  1.1667017e-02  1.5080884e+02]]
[[-180.   50.  -90.  180.  -90.   90.    0.   90.]
 [-180.   50.  -90.  180.  -30.   30.    0.   90.]
 [-180.   50.  -50.  100.  -90.   90.    0.  150.]]
recon_loss: 1.3982, kl_loss: 4656.3770, total_loss: 0.0237


In [84]:
model.eval()
with torch.no_grad():
    x_test_sample, y_test_sample = next(iter(dataloader)) # [B, M, 4], [B, 8]
    
    x_input = x_test_sample[0].unsqueeze(0).to(device) # [1, M, 4]
    y_truth = y_test_sample[0].cpu().numpy()          # [8]
    
    y_pred, _, _ = model(x_input) # [1, 8]
    y_pred = y_pred.squeeze().cpu().numpy()
    
    y_truth_deg = np.rad2deg(y_truth)
    y_pred_deg = np.rad2deg(y_pred)

    y_truth_deg[0:4] = -1 * (y_truth_deg[0:4] % 360)
    y_truth_deg[4:] = y_truth_deg[4:] % 360
    y_pred_deg[0:4] = -1 * (y_pred_deg[0:4] % 360)
    y_pred_deg[4:] = y_pred_deg[4:] % 360

    
    # print("Example from Test Set (in deg):")
    # print("---------------------------------")
    # print(f"JOINT         | TRUTH (min/max)  | PRED (min/max)")
    # print(f"Shoulder Abd  | {y_truth_deg[0]:.1f} / {y_truth_deg[4]:.1f}   | {y_pred_deg[0]:.1f} / {y_pred_deg[4]:.1f}")
    # print(f"Shoulder Flex | {y_truth_deg[1]:.1f} / {y_truth_deg[5]:.1f}  | {y_pred_deg[1]:.1f} / {y_pred_deg[5]:.1f}")
    # print(f"Shoulder Rot  | {y_truth_deg[2]:.1f} / {y_truth_deg[6]:.1f}   | {y_pred_deg[2]:.1f} / {y_pred_deg[6]:.1f}")
    # print(f"Elbow Flex    | {y_truth_deg[3]:.1f} / {y_truth_deg[7]:.1f}    | {y_pred_deg[3]:.1f} / {y_pred_deg[7]:.1f}")
    
    print("Example from Test Set (in deg):")
    print("---------------------------------")
    print(f"JOINT         | TRUTH (min/max)  | PRED (min/max)")
    print(f"Shoulder Abd  | {y_truth[0]:.1f} / {y_truth[4]:.1f}   | {y_pred[0]:.1f} / {y_pred[4]:.1f}")
    print(f"Shoulder Flex | {y_truth[1]:.1f} / {y_truth[5]:.1f}   | {y_pred[1]:.1f} / {y_pred[5]:.1f}")
    print(f"Shoulder Rot  | {y_truth[2]:.1f} / {y_truth[6]:.1f}   | {y_pred[2]:.1f} / {y_pred[6]:.1f}")
    print(f"Elbow Flex    | {y_truth[3]:.1f} / {y_truth[7]:.1f}   | {y_pred[3]:.1f} / {y_pred[7]:.1f}")

    mse_rad = np.mean((y_truth - y_pred)**2) # MSE in rad
    print(f"\nExample's MSE (deg^2): {mse_rad:.6f}")
    # mse_deg = np.mean((y_truth_deg - y_pred_deg)**2) # MSE in deg=
    # print(f"Example's MSE (deg^2): {mse_deg:.6f}")

Example from Test Set (in deg):
---------------------------------
JOINT         | TRUTH (min/max)  | PRED (min/max)
Shoulder Abd  | -180.0 / -30.0   | -180.5 / -30.0
Shoulder Flex | 50.0 / 30.0   | 50.2 / 29.9
Shoulder Rot  | -90.0 / 0.0   | -90.4 / -0.0
Elbow Flex    | 180.0 / 90.0   | 180.8 / 90.7

Example's MSE (deg^2): 0.208359
